In [ ]:
# Installing the Portkey AI gateway developed by the Portkey team
!pip install portkey-ai

from portkey import Portkey, LLMBase, Message, DefaultParams

In [ ]:
import os

os.environ["PORTKEY_API_KEY"] = ""
os.environ["OPENAI_API_KEY"] = ""
os.environ["ANTHROPIC_API_KEY"] = ""

In [ ]:
metadata = {
    "_environment": "production",
    "_prompt": "test",
    "_user": "user",
    "_organisation": "acme",
}

messages = [
    Message(role="system", content="You are a helpful assistant"),
    Message(role="user", content="What can you do?"),
]

pk_llm = Portkey(default_params=DefaultParams(
    metadata=metadata,
    max_tokens=50,
    stream=True,
    messages=messages
))


In [ ]:
openai_llm = LLMBase(provider="openai", model="gpt-3.5-turbo")
anthropic_llm = LLMBase(provider="anthropic", model="claude-2")

In [ ]:
# Fallback Scenario
response = pk_llm.chat_completion.with_fallbacks(llms=[openai_llm], stream=True)
for i in response:
    print(i.choices[0].get("delta", {}).get("content"), end="", flush=True)

In [ ]:
# LoadBalancing.
llm1 = LLMBase(provider="anthropic", model="claude-2", weight=0.6)
llm2 = LLMBase(provider="openai", model="gpt-3.5-turbo", weight=0.4)

print("Testing Fallback & Retry functionality:")
response = pk_llm.chat_completion.with_loadbalancing(llms=[llm1, llm2], stream=True)
for i in response:
    print(i.choices[0].get("delta", {}).get("content"), end="", flush=True)